In [1]:
import time

import neuroglancer
import numpy as np

class InteractiveInference(object):
    def __init__(self, cube):

        viewer = self.viewer = neuroglancer.Viewer()
        viewer.actions.add('move_left', self.move_cube_left)
        viewer.actions.add('move_down', self.move_cube_down)
        viewer.actions.add('move_right', self.move_cube_right)
        viewer.actions.add('move_up', self.move_cube_up)

        self.cube = cube
        self.off_x = 0
        self.off_y = 0
        self.off_z = 0
        self.grid_layer = GridLayer(self.viewer)
        self.cube_volm = neuroglancer.LocalVolume(data=self.cube, voxel_size=[10000, 10000, 10000], voxel_offset=[self.off_x, self.off_y, self.off_z])

        with viewer.config_state.txn() as s:
            s.input_event_bindings.data_view['shift+keyw'] = 'move_up'
            s.input_event_bindings.data_view['shift+keys'] = 'move_down'
            s.input_event_bindings.data_view['shift+keya'] = 'move_left'
            s.input_event_bindings.data_view['shift+keyd'] = 'move_right'

        with viewer.txn() as s:
            s.layers['image'] = neuroglancer.ImageLayer(
                source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD585_fullres')
            # Resets X/Y/Z plane orientation
            s.navigation.pose.orientation = [0,0,0,1]
            # Zooms out 
        #     s.navigation.zoomFactor = 10000 # If 4panel
            s.navigation.zoomFactor = 5000 # If xy

            # Resets 3D Viewer Orientation
            s.perspectiveOrientation = [0,0,0,1]
            # Zooms out
            s.perspectiveZoom = 75000
            
            s.layers['cube'] = neuroglancer.ImageLayer(source=self.cube_volm)
            # s.layers['points'] = neuroglancer.AnnotationLayer(voxel_size=[100000, 100000, 100000])

    def move_cube_left(self, action_state):

        self.off_x -= 10
        cube_volm = neuroglancer.LocalVolume(data=self.cube, voxel_size=[10000, 10000, 10000],
                                             voxel_offset=[self.off_x, self.off_y, self.off_z])
        cube_layer = neuroglancer.SegmentationLayer(source=cube_volm)
        with self.viewer.txn() as s:
            s.layers['cube'] = cube_layer

    def move_cube_right(self, action_state):
        self.off_x += 10
        cube_volm = neuroglancer.LocalVolume(data=self.cube, voxel_size=[10000, 10000, 10000],
                                             voxel_offset=[self.off_x, self.off_y, self.off_z])
        cube_layer = neuroglancer.SegmentationLayer(source=cube_volm)
        with self.viewer.txn() as s:
            s.layers['cube'] = cube_layer

    def move_cube_up(self, action_state):
        self.off_y -= 10
        cube_volm = neuroglancer.LocalVolume(data=self.cube, voxel_size=[10000, 10000, 10000],
                                             voxel_offset=[self.off_x, self.off_y, self.off_z])
        cube_layer = neuroglancer.SegmentationLayer(source=cube_volm)
        with self.viewer.txn() as s:
            s.layers['cube'] = cube_layer

    def move_cube_down(self, action_state):
        self.off_y += 10
        cube_volm = neuroglancer.LocalVolume(data=self.cube, voxel_size=[10000, 10000, 10000],
                                             voxel_offset=[self.off_x, self.off_y, self.off_z])
        cube_layer = neuroglancer.SegmentationLayer(source=cube_volm)
        with self.viewer.txn() as s:
            s.layers['cube'] = cube_layer
            
            
class GridLayer():
    def __init__(self, viewer, shape=(80, 60, 50),  vox_size=(100000, 100000, 100000)):
        self.vox_size = vox_size
        self.shape = shape
        self.jump_x = 15
        self.jump_y = 15
        self.jump_z = 20
        self.viewer = viewer
        self.grid_dict = {}
        self.triangle_index_to_grid_pts = {}

    def print_coordiante(self):
        if 'selectedAnnotation' in self.viewer.state.layers._layers[2]._json_data:
            print(self.grid_dict[self.viewer.state.layers._layers[2]._json_data['selectedAnnotation']])
            

In [2]:
def get_cube_test_volume():
    cube = np.zeros((100, 100, 100), dtype=np.uint8)

    cube[:, :, :10] = 1
    cube[:, :, 90:100] = 1

    cube[:10, :, :] = 1
    cube[90:100, :, :] = 1

    cube[:, :10, :] = 1
    cube[:, 90:100, :] = 1
    
    return cube

cube = get_cube_test_volume()

neuroglancer.set_server_bind_address(bind_address='0.0.0.0', bind_port=35000)

inf = InteractiveInference(cube)
viewer = inf.viewer
ip_name = 'ec2-52-53-248-156.us-west-1.compute.amazonaws.com'  ## Change for different machine
ip_name = '132.239.73.85'  ## Change for different machine

print('http://' + ip_name + ':' + viewer.get_viewer_url().split(':')[2])

http://132.239.73.85:35000/v/1e9f922244e63d3f9276491b43aee81f72637959/
